# [pwntools](https://docs.pwntools.com/en/stable/)
* Es gibt zwei Packete
    * `pwnlib`
    * `pwn` (umfangreicher)

In [4]:
from pwn import *

## [Getting Started – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html)
### [Verbindung herstellen – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#making-connections)
* Um ein Programm/Binary zu kompromittieren, muss man mit diesem „sprechen“ => `pwnlib.tubes`-Modul
    * Standard-Schnittstelle, um mit Prozessen, Sockets, [Serial Ports – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#serial-ports) (bspw. USB-Schnittstellen?), etc. zu kommunizieren, dh. für `I/O`-Operationen
    * [tubes – pwntools-Doku](https://pwntools.readthedocs.org/en/latest/tubes.html)
    * Man kann `tubes` verbinden und irgendetwas mit `Regex` machen
* Es gibt auch ein `ssh`-Modul: `pwnlib.tubes.ssh`, [SSH – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#secure-shell)

#### Internetverbindung: `pwnlib.tubes.remote`

In [ ]:
conn = remote('ftp.ubuntu.com', 21)
conn.recvline() # doctest: +ELLIPSIS <-- Was bedeutet das?
# b'220...'

In [ ]:
conn.send(b'USER anonymous\r\n')
conn.recvuntil(b' ', drop=True)
# b'331'
conn.recvline()
# b'Please specify the password.\r\n'
conn.close()

#### Listener

In [ ]:
l = listen()
r = remote('localhost', l.lport)
c = l.wait_for_connection()
r.send(b'hello')
c.recv()
# b'hello'

#### Interaktion mit Prozessen: `pwnlib.tubes.process`
* Mit manchen Prozessen kann man interagieren: `process(<Prozess>).interactive()`
    * bspw. öffnet `process("/bin/sh").interactive()` eine Shell

In [ ]:
sh = process('/bin/sh')  # oder python-Executable, dh. mit python-shebang am Anfang
# Dem Prozess „Input“ schicken
sh.sendline(b'sleep 3; echo hello world;')
# Ausgabe des Prozesses
sh.recvline(timeout=1)
# b''
o = sh.recvline(timeout=5)
# b'hello world\n'
sh.close()

### [Integer & Bytes – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#packing-integers)
* `p<Bit-Anzahl>` verpackt (en.: to `p`ack) Integer/Hex-Zahlen als `b-String`
* `u<Bit-Anzahl>` rechnet (en.: to `u`npack) `b-String`/Byte-Iterable zu Integer um

In [12]:
i = p8(16) == b"\x10"
j = u8(b"A") == 0x41
i == j

True

### Diverses
* `context.log_level = 'debug'` sorgt dafür, dass jegliche gesendete und erhaltene Daten ausgegeben werden

### TODO
* [Setting Target Architecture and OS – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#setting-the-target-architecture-and-os)
    * Ich glaube, das benötigt man, wenn man Betriebssystem-spezifischen Code, bspw. `Assembler`-Code, mit `pwntools` schreiben oder von einer Byte-Darstellung aus lesbar machen möchte
    * [Assembly and Disassembly – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#assembly-and-disassembly)
    * ...oder bei Integer-Darstellungen, da diese von `endian`, `bits` und `sign` abhängen
* [Misc Tools – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#misc-tools)
* [ELF Manipulation – pwntools-Doku](https://docs.pwntools.com/en/stable/intro.html#elf-manipulation)

## [GitHub-Tutorial](https://github.com/Gallopsled/pwntools-tutorial/blob/master/)
### [`tubes` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md)
* `tubes` übernehmen `I/O`-Operationen, dh. wenn von „Empfangen“ und „Senden“ die Rede ist, geht es um Ein- sowie Ausgabe, bspw. bei einem Prozess/Programm/Binary ([Verbindung herstellen – pwntools-Notebook](http://localhost:8888/notebooks/pwntools_Anleitung.ipynb#Verbindung-herstellen-%E2%80%93-pwntools-Doku))

#### [Empfangen - pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#receiving-data)
* `recv(<Anzahl Bytes>)`: Empfange `<Anzahl Bytes>` Bytes an Daten, wenn verfügbar
* `recvline()`: Emfpange Daten bis `\n`
* `recvuntil(<Trennzeichen>)`: Empfange Daten bis `<Trennzeichen>` (in `Bytes`, bspw. als `b`-String (es gibt aber noch andere Möglichkeiten)
* `recvregex(<Regex>)`: Empfange Daten bis `<Regex>` TODO: `<Regex>` in `Bytes`?
* `recvrepeat(<Zeitlimit in ?>)`: Empfange Daten entsprechend des Zeitlimits
* `clean()`: Verwerfe gepufferte Daten

#### [Senden – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#sending-data)
* `send(<Daten in Bytes>)`: Sendet Daten
* `sendline(<Zeile>)`: Verschickt Daten + `\n`

#### [Prozesse und deren Grundlagen – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#processes-and-basic-features)
* [`process` – pwntools-Doku](https://pwntools.readthedocs.org/en/latest/tubes/processes.html)-Objekt mit Name des Ziel-Binarys instanziieren

In [ ]:
io = process('sh')
io.sendline('echo Hello, world')
io.recvline()
# b'Hello, world\n'

* Kommandozeilenargumente und Umgebungsvariablen können auch gesetzt werden:

In [ ]:
io = process(['sh', '-c', 'echo $MYENV'], env={'MYENV': 'MYVAL'})
io.recvline()
# b'MYVAL\n'

#### [Netzwerkverbindungen – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md#networking)
* `remote`-Objekt stellt Verbindung *nach außen* her
    * per `typ=…` & `fam=…` können Protokollinformationen angegeben werden 
* `listen`-Instanz wartet auf Verbindung *von außen*

In [ ]:
io = remote('google.com', 80)
io.send('GET /\r\n\r\n')
io.recvline()
# 'HTTP/1.0 200 OK\r\n'

In [ ]:
client = listen(8080).wait_for_connection()

### [Ver- & Entpacken von Integern – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#packing-and-unpacking-integers)
* Umrechnen zwischen Byte-/Hex- und Basis 10-Darstellung von Integern
    * Funktionen beachten globale Einstellungen in [`context` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md)

### [Schreiben & Lesen von Dateien – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#file-io)
* `write(<Dateiname>, <Daten>)`: `<Daten>` können als `Bytes` oder `String` vorliegen
* `read(<Dateiname>, [<Anzahl Zeichen/Bytes>])`: `<Anzahl Zeichen/Bytes>` optional, wenn nicht spezifiziert, wird ganze Datei gelesen

### [Hashing & Encoding – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#hashing-and-encoding)
* Base64: `b64e(<Bytes>)`, `b64d<String/Bytes>)`
* md5: `md5(sum|file)hex(<Bytes/Dateiname>)`: Berechnet `md5`-Hash; `sum` für `Byte`-Objekt, `file` um Dateiinhalt (muss nicht in `Bytes` vorliegen) zu hashen
* sha1: `sha1(sum|file)hex(<Bytes/Dateiname>)`: Analog zu md5
* URL-Encoding: `urlencode(<String>)` (funktioniert nicht mit `Bytes`
* String zu Hex: `enhex(<Bytes>)` und `unhex(<Hex-Zahl>)`
    * `enhex(<Bytes>)` macht dasselbe wie `hex(b) for b in <Bytes>`
* [Bits – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#bit-manipulation-and-hex-dumping): `bits(<Zahl>)`, wobei `<Zahl>` alles mögliche sein kann, dh `int`, `0x…`, `0b…` usw.  
    * Rückgabe: Liste mit den `Bits`, zB `bits(0x8) == [0, 0, 0, 0, 1, 0, 0, 0]`
* [Hex-Dumping – python-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#hex-dumping): TODO Unklar, was das genau ist/macht und wofür es gut ist  
`hexdump(…)`

### [Puffer-Überlauf und Mustergenerierung](https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md#pattern-generation)
* TODO Ich weiß zwar, was ein Puffer-Überlauf (Buffer-Overflow) ist, aber aus dem Stand heraus weiß ich gerade nicht, wie ich diese Funktionalität anwenden kann, da mir die Praxis fehlt

## [`Context`-Objekt – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#context)
* globales, „Thread-awares“ Objekt, das verschiedenste Einstellung enthält
* Man findet am Skript-Anfang häufig etwas wie `context.arch = 'amd64`, um anzugeben für welche Architektur der Shellcode generiert werden soll
* [`arch` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#arch): `aarch64`, `arm`, `i386` (Standard), `amd64`
* [`bits` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#bits): `32` oder `64` Bit
* [`binary` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#binary): Übernimmt Einstellungen aus einer ELF-Datei TODO Unklar, was das genau macht
* [`endian` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#endian): `big` oder `little` (Standard)
* [`log_file` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#log_file): Datei in der die Log-Informationen geschrieben werden sollen
* [`log_level` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#log_level): Granularität des Loggings; Integer- oder String-Werte (wie `debug`, `info` oder `error`)
* [`sign` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#sign): Verwendung von Vorzeichen-behafteten Integern oder nicht für [`pack` und `unpack` – pwntools-Notebook](http://localhost:8888/notebooks/pwntools_Anleitung.ipynb#Ver--&-Entpacken-von-Integern-%E2%80%93-pwntools-GitHub); `signed` oder `unsigned` (Standard)
* [`terminal` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#terminal): Welches Terminal beim Öffnen neuer Fenster verwendet werden soll; Standard: `x-terminal-emulator` oder `tmux`
* [`timeout` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#update): Standardzeitlimit für [`tube` – pwntools-Notebook](http://localhost:8888/notebooks/pwntools_Anleitung.ipynb#tubes-%E2%80%93-pwntools-GitHub)-Operationen
* [`update` – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/context.md#update): Setzt mehrere Werte auf einmal, bspw. `context.update(arch='mips', bits=64, endian='big')`

### [ELF-Dateien – pwntools-GitHub](https://github.com/Gallopsled/pwntools-tutorial/blob/master/elf.md)
* `ELF`-Dateien sind „Executable und Linkable Files“
* Ab hier fortsetzen